<br>
#  <strong>알고리즘 트레이딩 성능평가와 최적화</strong> 

<strong>금융경제의 불확실성응로 인해, 사후관리 또한 중요하다
1. 알고리즘 트레이딩 시스템의 성능 측정
1. 알파모델을 선택시 유의할 점
1. 수익률 관리를 좌우하는 '파라미터 최적화' 방법

<br>
## <strong>1 알고리즘 트레이딩 시스템의 성능측정
1. 구현 시스템의 수익성 평가
1. 구현 모델별 비교 
    1. 시스템에 대한 자신감 : 어느 시스템도 첫날부터 만족할 수익을 내지 않으므로 불완전 모델을 완성시켜가는 과정으로써 중요하다
    1. 추세추종 기법의 경우 많은 거래시 손실을 발생하지만, 몇번의 큰 수익의 기회를 준다

<br>
## <strong>2 백 테스팅
특정기간의 history 데이터로 시스템 성능을 평가하는 것으로써, 예측력과 수익을 측정하는 중요한 수단이 된다
1. Profit/Loss (손익/수익률)
1. Hit raito (적중률)
1. Draw_down (고점대비 최대 손실률)
1. Sharp Raito (샤프지수) 가 있다

### <strong>01 Profit / Loss 테스트
특정기간 데이터를 입력하여 트레이등의 수익률을 평가한다

<img src="https://www.rita.dot.gov/bts/sites/rita.dot.gov.bts/files/publications/special_reports_and_issue_briefs/special_report/2012_03_33/images/figure_03.gif" align="left">

### <strong>02 Hit Ratio
모델을 이용하여 예측한 결과가 얼마나 적합한가를 판단한다

In [4]:
# 일정기간 특정종목의 예측 정확도를 계산
def getHitRatio(self ,name ,code ,start_date ,end_date ,lags_count=5 ):
    a_predictor = self.predictor.get(code, name)
    df_dataset  = self.predictor.makeLaggedDataset(code ,
                        start_date, end_date ,
                        self.config.get('input_column'),
                        self.config.get('output_column'), lags_count)
    df_x_test = df_dataset[[self.config.get('input_column')]]
    df_y_true = df_dataset[[self.config.get('output_column')]].values
    df_y_pred, df_y_pred_probability = a_predictor.predict(df_x_test)
    hit_count = 0
    total_count = len(df_y_true)
    for index in range(total_count):
        if(df_y_pred[index] == df_y_true[index]):
            hit_count = hit_count + 1
    hit_ratio = hit_count / total_count
    print ("hit_count = %s, total=%s, hit_ratio=%s"%(hit_count ,total_count, hit_ratio))
    return hit_ratio

In [1]:
# 예측 결과에 따라 매수/매도 포지션에 따른 'Yes'를 출력한다
def drawHitRatio( self, name, code, start_date, end_date, lags_count = 5):
    a_predictor = self.predictor.get(code ,name)
    df_dataset  = self.predictor.makeLaggedDataset(code, start_date, end_date ,
                  self.config.get('input_column'),
                  self.config.get('output_column'),lags_count)
    df_x_test = df_dataset[[ self.config.get('input_column')]]
    df_y_true = df_dataset[[ self.config.get('output_column')]].values
    df_y_pred , df_y_pred_probability = a_predictor.predict(df_x_test)
    ax = df_dataset[[self.config.get('input_column')]].plot()
    for row_index in range( df_y_true.shape[0]):
             if(df_y_pred[row_index] == df_y_true[row_index]):
                 ax.annotate('Yes', xy=(row_index ,df_dataset.loc[row_index ,
                                    self.config.get('input_column')]),
                                    xytext = ( 10 , 30 ),
                                    textcoords = 'offset points', 
                                    arrowprops = dict ( arrowstyle = '-|>'))

### <strong>03 Draw down
Hit Ratio가 적합성을 판단한다면, Draw down은 잘못된 예측이 얼나마되며 그 기간이 얼마인지를 알려준다
1. Maximum Drawdown : 최대손실
1. Drawdown Duration : 손실지속기간
1. Drawdown Curve : 손실경향으로 시스템의 손실 특성을 파악할 수 있다

### <strong>04 Sharp Ratio (위험을 고려한 성능을 나타내는 수치)
<strong>$ Sharp Ratio = \frac{E(R-R_b)}{\sqrt(R-R_b)} $는 '기대값(수익률) / 기대값 표준편차(변동성)'</strong> 를 계산한 결과로 

Reward/Risk 분석이라고도 부른다.

수익률이 높아도, 변동성 또한 높으면 위험이 커져서 지속적인 운영의 문제가 생기므로

Sharp Ratio(수익률/변동성)를 활용하면 객관적 안정적 모델 평가가 가능하다 

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d7/Sharpe_ratio_graph.jpg" align='left' width='300'>

<br>
## <strong>3 머신러닝 모델의 성능측정
Hit Ratio, Risk/Reward 테스트는 성능은 측정가능하나, 모델 자체의 평가는 아니다
1. 회귀모델 : <strong>제곱근 평균 제곱오차(Root Mean Square Error)</strong>등의 예측값과 실측값의 차이를 통해서 판단
2. 분류모델 : <strong>혼동행렬(Confusion Matrix)</strong> 같은 항목별 정확도/ 재현율 특성을 계산

### <strong>01 혼동행렬(Confusion Matrix)
http://bcho.tistory.com/tag/Confusion%20matrix

예측특성과 데이터 편향등의 추가적 정보를 확인 가능하다

<img src="https://thebook.io/img/006723/447.jpg" align='left' width='500'>

### <strong>02 Classification Report
sci-kit learn의 기본모듈로써, 학습시킨 모델의 성능을 알기쉽게 자동으로 요약한다
1. 정확도 Precision : 모델의 정확성
1. 재현율 Recall : 다른 데이터셋에서도 정확하게 예측할 확률
1. 특이도 Specificity : Recall과 반대로 Negative를 정확하게 예측할 확률
1. f1-score : Precision과 recall의 조화평균값

<img src="http://www2.isprs.org/tl_files/isprs/wg34/images/Precision_Recall_formula.png" align='left' width='200'>

### <strong>03 ROC curve (Receiver Operatin Characteristic Curve)
ROC 커브는 Recall 성능이, Specificity에 따라 어떻게 달라지는지를 시각적으로 보여준다
1. <strong>X축</strong>은 <strong>Specificity</strong>를
1. <strong>Y축</strong>은 <strong>Recall</strong>를 나태냄으로써 시각적으로 모델성능을 평가 가능하다
1. <strong>AUC (area Under Curve)</strong>는 <strong>ROC 곡선의 면적</strong>으로써, 값이 클수록 우수한 성능을 갖는다
1. 맨 아래의 직선은 random분류를, 맨 위의 직선은 정확도1인 분류 모델로써, <strong>AUC 값이 클수록</strong> 높은 성능을 갖는다

<img src="https://www.researchgate.net/profile/Claes_Norring/publication/269030748/figure/fig1/AS:295435752230920@1447448780827/Fig-1-Receiver-operating-characteristic-ROC-curve-for-anorexia-nervosa-patients-AUC.png" align='left' width='250'>

In [6]:
def drawROC(self ,y_true ,y_pred):
    false_positive_rate, true_positive_rate ,thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC=%0.2f'%roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1 ,1.2])
    plt.ylim([-0.1 ,1.2])
    plt.ylabel('Sensitivity')
    plt.xlabel('Specificity')
    plt.show()

<br>
## <strong>4 Live trading Monitoring 
실시간 알고리즘 트레이딩 시스템의 매수/매도를 모니터링 하는 Dash Board

<br>
## <strong>5 파라미터 최적화/ 하이퍼 파라미터 최적화
알파모델의 성능을 극대화 및 지속을 위해선 중요
1. <strong>격자탐색 (Grid Search)</strong> : 주어진 범위내 입력값을 모두 대입하여 최적의 해를 찾는다 (최적해 도출)
1. <strong>랜덤탬색 (Random Search)</strong> : 주어진 값에서 표본을 추출 후, 표본범위에서 변수를 찾는다 (최선해 도출)
    1. 60개 이상의 표본을 반복하므로 격자탐색에 비해 성능이 떨어지진 않는다